<a href="https://colab.research.google.com/github/thyelleteruya/Python1/blob/main/Exerc%C3%ADcio_Aula_6_Portal_da_Transpar%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1 - Configurando o Spark no Python**

Etapa responsável por realizar o Download e configuração do Apache Spark.

In [2]:
#Instacao e configuracao do Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Download do Apache Spark
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#Descompatacao do Apache Spark
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#Instalacao do Findspark
!pip install -q findspark

**2 - Configurando o Ambiente**

Etapa responsável por configurar o ambiente do Spark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

**3 - Configurando a sessão do Spark**

Etapa responsável por inicializar a sessão do Spark

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

**4 - Validando a Configuração e Versão do Spark**

Etapa de verificação 

In [5]:
import pyspark
print(pyspark.__version__)

3.0.1


**5 - Download dos Dados do Portal da Transparência**

Etapa responsável por realizar o Download das informações de Gastos por meio de cartão de pagamento providas através da API: http://www.portaltransparencia.gov.br/api-de-dados/cartoes?pagina=1

In [16]:
!curl -X GET --header 'Accept: application/json' --header 'chave-api-dados: 43a93a2a4797fad79da698cd394ca2c8' 'http://www.portaltransparencia.gov.br/api-de-dados/cepim?pagina=1' > consulta_portal_transparencia.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26440    0 26440    0     0  17828      0 --:--:--  0:00:01 --:--:-- 17828


**6 - Carregando os dados no Spark**

Etapa paga a carga de dados no Spark.

In [17]:
#Instancia o contexto do Spark
sc = spark.sparkContext
#Realiza a leitura da nossa consulta realizada na API do Governo Federal 
path = "/content/consulta_portal_transparencia.json"
#Declara dataframe df com o conteudo da api que estava no JSON
df = spark.read.json(path)

**7 - Explorando os dados que foram obtidos**

Etapa de exploração dos dados.

In [18]:
#Imprime a estrutura de dados do nosso dataframe
df.printSchema()
#Imprime uma amosta de dados ( para especificar as linhas basta colocar a quantidade entre ())
df.show()

root
 |-- convenio: struct (nullable = true)
 |    |-- codigo: string (nullable = true)
 |    |-- numero: string (nullable = true)
 |    |-- objeto: string (nullable = true)
 |-- dataReferencia: string (nullable = true)
 |-- id: long (nullable = true)
 |-- motivo: string (nullable = true)
 |-- orgaoSuperior: struct (nullable = true)
 |    |-- cnpj: string (nullable = true)
 |    |-- codigoSIAFI: string (nullable = true)
 |    |-- descricaoPoder: string (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- orgaoMaximo: struct (nullable = true)
 |    |    |-- codigo: string (nullable = true)
 |    |    |-- nome: string (nullable = true)
 |    |    |-- sigla: string (nullable = true)
 |    |-- sigla: string (nullable = true)
 |-- pessoaJuridica: struct (nullable = true)
 |    |-- cnae: struct (nullable = true)
 |    |    |-- classe: string (nullable = true)
 |    |    |-- codigoClasse: string (nullable = true)
 |    |    |-- codigoDivisao: string (nullable = true)
 |    |  

**8 - Tratando e transformando os dados**

In [19]:
dfPrepared = df.selectExpr("motivo","dataReferencia","orgaoSuperior")
dfPrepared.show()

+--------------------+--------------+--------------------+
|              motivo|dataReferencia|       orgaoSuperior|
+--------------------+--------------+--------------------+
|NAO APRESENTACAO ...|    21/10/2020|[00394494000136, ...|
|IRREGULARIDADE NA...|    21/10/2020|[00394494000136, ...|
|INSTAURACAO DE TO...|    21/10/2020|[00394494000136, ...|
|MOTIVO NÃO ESPECI...|    21/10/2020|[00394494000136, ...|
|INSTAURACAO DE TO...|    21/10/2020|[00394494000136, ...|
|INSTAURACAO DE TO...|    21/10/2020|[00394494000136, ...|
|INSTAURACAO DE TO...|    21/10/2020|[00394494000136, ...|
|NAO APRESENTACAO ...|    21/10/2020|[00394494000136, ...|
|INSTAURACAO DE TO...|    21/10/2020|[00394494000136, ...|
|NAO EXECUTOU TOTA...|    21/10/2020|[00394494000136, ...|
|IRREGULARIDADE NA...|    21/10/2020|[00394494000136, ...|
|IRREGULARIDADE NA...|    21/10/2020|[00394494000136, ...|
|IRREGULARIDADE NA...|    21/10/2020|[00394494000136, ...|
|IRREGULARIDADE NA...|    21/10/2020|[00394494000136, ..

**9 - Analise das informações e resultados obtidos**

Etapa onde são realizadas as analises utilizando linguagem SQL para por exemplo descobrir os maiores motivos de impedimentos das Entidades Privadas sem Fins Lucrativos.

In [22]:
#Declara tabela temporaria no SparkSQL
dfPrepared.createOrReplaceTempView("dfPrepared")

In [23]:
topTravels = spark.sql("select count(*),motivo from dfPrepared group by 2 order by 1 desc")
topTravels.show()

+--------+--------------------+
|count(1)|              motivo|
+--------+--------------------+
|       5|INSTAURACAO DE TO...|
|       3|IRREGULARIDADE NA...|
|       2|MOTIVO NÃO ESPECI...|
|       2|IRREGULARIDADE NA...|
|       1|NAO APRESENTACAO ...|
|       1|NAO APRESENTACAO ...|
|       1|NAO EXECUTOU TOTA...|
+--------+--------------------+

